<a href="https://colab.research.google.com/github/jsKim-prog/pythonStudy23_JS/blob/master/Colab_5%EC%A3%BC%EC%B0%A802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas-datareader finance-datareader

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr

In [ ]:
def get_return_mf(code):
  df = fdr.DataReader(code,'2018')

  df = df.reset_index()

  df = df[['Date','Open']]

  df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

  cond = (df['Day'] == 'Friday') | (df['Day'] == 'Monday')
  df = df[cond]

  if df.iloc[0,2] == 'Friday':
    df = df.drop(index=df.index[0])

  if df.iloc[-1,2] == 'Monday':
    df = df.drop(index=df.index[-1])

  cond = (df['Day'] == 'Monday') & (df['Day'].shift(-1) == 'Monday')
  df = df.drop(index= df[cond].index)

  cond = (df['Day'] == 'Friday') & (df['Day'].shift(1) == 'Friday')
  df = df.drop(index= df[cond].index)

  df['Open_fri'] = df['Open'].shift(-1)

  df = df[df['Day']=='Monday']
  df = df[['Open','Open_fri']]

  df.columns = ['buy_at','sell_at']

  df['return'] = df['sell_at'] / df['buy_at']

  cond = (df['sell_at']!=0) & (df['buy_at']!=0)
  df = df[cond]

  return df[['return']].cumprod().iloc[-1,-1]-1

In [ ]:
!pip install dart-fss

In [ ]:
import dart_fss as dart_fss
import pandas as pd

api_key = 'bb5c1988eeec4920f72010e37e3b26fa2e1d6602'
dart_fss.set_api_key(api_key=api_key)

all = dart_fss.api.filings.get_corp_code()

df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]

In [ ]:
df = pd.DataFrame()
for row in df_listed.sample(10)[['stock_code','corp_name']].itertuples():
  try:
    doc = {
        'name' : row[2],
        'return' : get_return_mf(row[1])
    }
    df = df.append(doc,ignore_index=True)
  except:
    print(f'error-{row[2]}')

df.sort_values(by='return',ascending=False)

<ipython-input-39-c12adad89f4d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc,ignore_index=True)
<ipython-input-39-c12adad89f4d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc,ignore_index=True)
<ipython-input-39-c12adad89f4d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc,ignore_index=True)
<ipython-input-39-c12adad89f4d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc,ignore_index=True)
<ipython-input-39-c12adad89f4d>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


error-대선조선
error-모아에스앤에스


,name,return
3,우리금융캐피탈,1.008214
7,하나투어,0.123971
6,미쥬,0.018393
5,엔피케이,-0.191800
1,모니터랩,-0.522919
4,한독,-0.653997
0,케일럼,-0.827145
2,엄지하우스,-0.859215
